# Model Training for Hand Gesture Recognition

This notebook is used for training the model to recognize five American Sign Language (ASL) signs. It includes steps for loading the dataset, training the model, and evaluating its performance.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import re

# Set paths
data_dir = r"E:\new yousef\hand-gesture-recognition\data\processed"
model_save_path = r"E:\new yousef\hand-gesture-recognition\models\asl_gesture_recognition_model.h5"
img_height = 64
img_width = 64
batch_size = 64
epochs = 20
num_classes = 5

# Define class names
class_names = ['hello', 'yes', 'no', 'i love you', 'thank you']
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

# Custom generator to load images in batches
def data_generator(data_dir, batch_size, img_height, img_width, class_to_idx, shuffle=True):
    # Filter valid filenames upfront
    filenames = []
    for f in os.listdir(data_dir):
        if not f.endswith('.png'):
            continue
        sign_name = re.match(r'([^_]+)_processed_', f)
        if sign_name and sign_name.group(1) in class_names:
            filenames.append(f)
        else:
            print(f"Skipping invalid filename: {f}")
    
    if not filenames:
        raise ValueError(f"No valid images found in {data_dir} with expected naming pattern '<sign_name>_processed_...'")

    if shuffle:
        np.random.shuffle(filenames)
    
    i = 0
    while True:
        batch_images = []
        batch_labels = []
        attempts = 0
        max_attempts = len(filenames)  # Prevent infinite loop
        
        while len(batch_images) < batch_size and attempts < max_attempts:
            if i >= len(filenames):
                i = 0
                if shuffle:
                    np.random.shuffle(filenames)
            
            filename = filenames[i]
            img_path = os.path.join(data_dir, filename)
            try:
                img = load_img(img_path, target_size=(img_height, img_width))
                img_array = img_to_array(img) / 255.0
            except Exception as e:
                print(f"Warning: Failed to load image '{img_path}'. Skipping. Reason: {e}")
                i += 1
                attempts += 1
                continue
            
            label_idx = class_to_idx[re.match(r'([^_]+)_processed_', filename).group(1)]
            batch_images.append(img_array)
            batch_labels.append(label_idx)
            i += 1
            attempts += 1
        
        if not batch_images:
            print("No valid images could be loaded for this batch. Stopping generator.")
            raise StopIteration
        
        batch_images = np.array(batch_images)
        batch_labels = tf.keras.utils.to_categorical(batch_labels, num_classes=len(class_names))
        yield batch_images, batch_labels

# Count total images for steps per epoch
valid_filenames = [f for f in os.listdir(data_dir) if f.endswith('.png') and re.match(r'([^_]+)_processed_', f) and re.match(r'([^_]+)_processed_', f).group(1) in class_names]
total_images = len(valid_filenames)
if total_images == 0:
    print(f"No valid images found in {data_dir}")
    exit()

# Define steps per epoch and validation steps
train_split = 0.8
steps_per_epoch = max(1, int(total_images * train_split) // batch_size)
validation_steps = max(1, int(total_images * (1 - train_split)) // batch_size)

# Create generators
train_generator = data_generator(data_dir, batch_size, img_height, img_width, class_to_idx, shuffle=True)
val_generator = data_generator(data_dir, batch_size, img_height, img_width, class_to_idx, shuffle=False)

# Define the model using MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (initial training)
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs // 2,
    validation_data=val_generator,
    validation_steps=validation_steps,
    verbose=1
)

# Fine-tune the model
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs // 2,
    validation_data=val_generator,
    validation_steps=validation_steps,
    verbose=1
)

# Save the model
try:
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")
except PermissionError:
    print(f"Error: No permission to save model to '{model_save_path}'. Try running as administrator or checking folder permissions.")
    exit()
except OSError as e:
    print(f"Error: Failed to save model to '{model_save_path}'. Reason: {e}")
    exit()

# Evaluate the model
def evaluate_model(model, data_dir, batch_size, img_height, img_width, class_to_idx, class_names, num_samples=5000):
    val_gen = data_generator(data_dir, batch_size, img_height, img_width, class_to_idx, shuffle=False)
    y_true, y_pred = [], []
    
    samples_processed = 0
    for batch_images, batch_labels in val_gen:
        batch_pred = model.predict(batch_images, verbose=0)
        y_true.extend(np.argmax(batch_labels, axis=1))
        y_pred.extend(np.argmax(batch_pred, axis=1))
        samples_processed += len(batch_images)
        if samples_processed >= num_samples:
            break
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation Precision: {precision:.4f}")
    print(f"Validation Recall: {recall:.4f}")
    print(f"Validation F1-Score: {f1:.4f}")
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.close()

evaluate_model(model, data_dir, batch_size, img_height, img_width, class_to_idx, class_names)

# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
train_acc = history.history['accuracy'] + fine_tune_history.history['accuracy']
val_acc = history.history['val_accuracy'] + fine_tune_history.history['val_accuracy']
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
train_loss = history.history['loss'] + fine_tune_history.history['loss']
val_loss = history.history['val_loss'] + fine_tune_history.history['val_loss']
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.savefig('training_plots.png')
plt.close()

C:\Users\pc\AppData\Local\Temp\ipykernel_1780\1535413110.py:100: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 46s 127ms/step - accuracy: 0.9557 - loss: 0.1269 - val_accuracy: 1.0000 - val_loss: 4.4562e-04
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 35s 114ms/step - accuracy: 0.9994 - loss: 0.0026 - val_accuracy: 1.0000 - val_loss: 4.5141e-04
Epoch 3/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 35s 112ms/step - accuracy: 0.9991 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 7.6488e-05
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 36s 115ms/step - accuracy: 0.9993 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 5.7710e-06
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 36s 114ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 1.8143e-05
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 36s 116ms/step - accuracy: 1.0000 - loss: 3.9804e-04 - val_accuracy: 1.0000 - val_loss: 4.6614e-05
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 36s 116ms/step - accuracy: 0.9998 - loss: 8.1221e-04 - val_accuracy: 1.0000 - val_loss: 9.2756e-05
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 36s 116

Model saved to E:\new yousef\hand-gesture-recognition\models\asl_gesture_recognition_model.h5
Validation Accuracy: 1.0000
Validation Precision: 1.0000
Validation Recall: 1.0000
Validation F1-Score: 1.0000
